In [1]:
# !pip install pandas
# !pip install ollama
# !pip install faiss-cpu
# !pip install numpy
# !pip install sentence_transformers
# !pip install langchain
# !pip install -U langchain-ollama

In [2]:
# pip install langchain_community


In [3]:
import pandas as pd
import ollama
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
import langchain
from langchain.llms import Ollama
from random import randint
from tqdm import tqdm

/opt/anaconda3/envs/purple/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Retriever

In [101]:
model = ollama.Client()


In [102]:
def generate_response(prompt):
    response = model.generate(model='gemma2', prompt=prompt)
    return response['response']

In [103]:
%%time

generate_response('Напиши 3 случайных предложения')

CPU times: user 2.6 ms, sys: 6.57 ms, total: 9.17 ms
Wall time: 4.07 s


'1. Кошка аккуратно запрыгнула на подоконник, наблюдая за падающими листьями.\n2. Звуки джазовой музыки доносились из открытой двери кафе.\n3. Внезапный шторм захлестнул город, оставив после себя мокрые улицы и шум ветра. \n\n\n'

# Generate categories from user's query

## Dialogue

In [9]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [48]:
# Конфигурация памяти для хранения истории диалога
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="user_input"
)

# Шаблон промта с поддержкой контекста
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """Вы ассистент по подбору еды. Отвечайте дружелюбно, учитывая историю диалога."""),
    MessagesPlaceholder(variable_name="history"), ("human", "{user_input}")
])

# Создание цепочки обработки
chain = LLMChain(
    llm=model,
    prompt=prompt_template,
    memory=memory,
    verbose=True
)

In [49]:
# Первый запрос пользователя
response = chain.run(user_input="Привет! Есть что-нибудь сладкое?")
print(f"Ассистент: {response}")
# Output: "Привет! Что вы хотите? Тортик, снеки?"




> Entering new LLMChain chain...
Prompt after formatting:
System: Вы ассистент по подбору еды. Отвечайте дружелюбно, учитывая историю диалога.
Human: Привет! Есть что-нибудь сладкое?

> Finished chain.
Ассистент: Привет! 😊  Конечно есть!   Что тебе больше нравится - шоколадное пирожное, ароматный кекс или может быть свежие ягоды с кремом?  🍓🍰🍫 





In [32]:
# Второй запрос (контекст сохраняется)
response = chain.run(user_input="Торт")
print(f"Ассистент: {response}")
# Output: "Могу предложить медовик или наполеон. Что предпочитаете?"



> Entering new LLMChain chain...
Prompt after formatting:
System: Вы ассистент по подбору еды. Отвечайте дружелюбно, учитывая историю диалога.
Human: Привет! Есть что-нибудь сладкое?
AI: Привет!  Конечно есть! Что ты любишь? 🍰🍫🍦  Шоколадный торт, мороженое или может быть чего-то полегче - печенье с малиновым джемом? 😊 Расскажи мне о своих предпочтениях, и я подберу что-нибудь вкусненькое. 💕  

Human: Торт

> Finished chain.
Ассистент: Отличный выбор! 🎂 Какой именно торт ты предпочитаешь? Шоколадный, вишневый, или может быть с фруктами? 😋 





In [33]:
# Третий запрос с уточнением
response = chain.run(user_input="С кремом")
print(f"Ассистент: {response}")
# Output: "Наполеон с заварным кремом. Хотите оформить заказ?"



> Entering new LLMChain chain...
Prompt after formatting:
System: Вы ассистент по подбору еды. Отвечайте дружелюбно, учитывая историю диалога.
Human: Привет! Есть что-нибудь сладкое?
AI: Привет!  Конечно есть! Что ты любишь? 🍰🍫🍦  Шоколадный торт, мороженое или может быть чего-то полегче - печенье с малиновым джемом? 😊 Расскажи мне о своих предпочтениях, и я подберу что-нибудь вкусненькое. 💕  

Human: Торт
AI: Отличный выбор! 🎂 Какой именно торт ты предпочитаешь? Шоколадный, вишневый, или может быть с фруктами? 😋 



Human: С кремом

> Finished chain.
Ассистент: Отличная идея! 🍰💖   А какой крем тебе нравится больше - сливочный, ванильный,  белый шоколад? 🤔  Я могу предложить тебе что-нибудь по нашему каталогу тортов с кремом.





## Final

In [106]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from search_engine import FaissSearchEngine
from typing import List, Dict, Optional


In [107]:
items_bd = pd.read_csv('../../Downloads/base_ua_db.csv')
items_bd.head()

,Unnamed: 0,country_code,city_code,store_name,product_name,collection_section,product_description,subset,hash,aux_store,HIER,s3_path,full_name,id
0,753,UA,MPL,Mikado,Рол Кампай (220 г),РОЛИ DELUX,"Угорь, лосось, том ям, огурец, лист салата, сы...",train,3903697417650224690,False,True,dataset/BDKZJSR_0000806_2000078286.png,"Рол Кампай (220 г) \n Угорь, лосось, том ям, о...",25497
1,5766,UA,MPL,New York Street Pizza,Тотори Сикору (220 г),Эти роли,"Морской гребешок, сыр для суши, огурец, икра к...",train,-3451380953300451258,False,True,dataset/BDKZJSR_0000503_1927787986.png,"Тотори Сикору (220 г) \n Морской гребешок, сыр...",85908
2,6601,UA,MPL,New York Street Pizza,Салат Цезарь с лососем (200 г),Салаты и антипасты,"Салат из разных овощей, слабосоленый лосось, п...",train,-3276916747514226914,False,True,dataset/BDKZJSR_0000423_1927638127.png,Салат Цезарь с лососем (200 г) \n Салат из раз...,12266
3,7153,UA,MPL,Detroit. City diner,Рыба и чипсы (280 г),На старт!,С чесночным йогуртом,train,-991058856461244767,False,True,dataset/BDKZJSR_0000538_1944956442.png,Рыба и чипсы (280 г) \n С чесночным йогуртом,53171
4,7771,UA,MPL,Mafia / Мафія,"Газировка «Моршинська» (0,75 л)",Напитки,Минеральная газированная вода,train,-8606000751944571868,False,True,dataset/BDKZJSR_0000103_1882646104.png,"Газировка «Моршинська» (0,75 л) \n Минеральная...",20355


In [108]:
MODEL_PATH = '../models/rubert-tiny-turbo'
INDEX_PATH = 'faiss_index.bin'

search_engine = FaissSearchEngine(model_path=MODEL_PATH)
search_engine.load_index(INDEX_PATH)

Index and data loaded successfully


In [109]:
items_bd.iloc[0]['id']

np.int64(25497)

In [110]:
class ShoppingAssistant:
    def __init__(self, model, items_bd: pd.DataFrame, search_engine: FaissSearchEngine):
        self.llm = model
        self.items_bd = items_bd
        self.search_engine = search_engine

        # истории диалога
        self.memory = ConversationBufferMemory() 
        self.readonly_memory = ReadOnlySharedMemory(memory=self.memory)
        
        # id текущих товаров в корзине пользователя
        self.current_product_ids: List[int] = []
        
        # Агент для генерации категорий
        self.category_agent = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["history", "input"],
                template="""Ты помощник по подбору еды. Предложи продукты, которые нужны пользователю, исходя из его запроса
                
                История диалога:
                {history}
                
                Запрос: {input}
                Категории (только названия через запятую):"""
            ),
            memory=self.memory,
            output_key="categories"
        )
        
        # Агент для редактирования списка
        self.editor_agent = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["current_products", "input", "history"],
                template="""Тебе будет дан список товаров и комментарий пользователя. Отредактируй список товаров на основе комментария пользователя.
                Если товар менять не надо, выведи его название без изменений.
                Если пользователь хочет что-то другое, замени товар.
                Если пользователь просит что-то добавить, добавь новый товар.
                
                История диалога:
                {history}

                Текущие товары:
                {current_products}
                
                Запрос: {input}
                Новый список товаров (только названия, каждое с новой строки):"""
            ),
            memory=ConversationBufferMemory(
                memory_key="history",
                input_key="input"  # Явное указание ключа для памяти
            ),
            output_key="updated_products"
        )

        # агент для выбора лучшего товара
        self.best_item_agent = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["relevant_products", "input", "history"],
                template="""Тебе будет дан список товаров и запрос пользователя.
                Выбери ОДИН лучший товар на основе запроса пользователя и выведи его id.
                Выводи только id лучшего товара и больше ничего.
                Список товаров: {relevant_products}
                
                История диалога:
                {history}
                
                Запрос: {input}
                """
            ),
            memory=self.readonly_memory,
            output_key="best_product_id"
        )

        # агент для аргументации выбора
        self.explainer_agent = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["best_products", "input", "history"],
                template="""Ты очень доброжелательный и дружелюбный ассистент, помогающий пользователю с выбором продуктов.
                Тебе будет дан список товаров и запрос пользователя.
                Объясни пользователю, почему эти товары подходят ему лучше всего исходя из его запроса в 1 - 2 предложениях.
                Список товаров: {best_products}
                История диалога:
                {history}
                Запрос: {input}"""
            ),
            memory=self.readonly_memory,
            output_key="explanation"
        )

    def get_relevant_pruduct_id(self, user_input: str, product: str) -> str:
        relevant_products = search_engine.search(product, top_n=10)
        relevant_products =  ' \n '.join(list(map(str, relevant_products)))

        best_product = self.best_item_agent({
            "input": user_input,
            "history": self.memory.load_memory_variables({})["history"],
            'relevant_products': relevant_products
        })

    
        return best_product['best_product_id'].strip()
    
    def get_product_name_by_id(self, ids: List[int]) -> List[int]:
        return list(self.items_bd[items_bd['id'].isin(ids)]['product_name'].values)
    
    def get_full_name_by_id(self, ids: List[int]) -> List[int]:
        return list(self.items_bd[items_bd['id'].isin(ids)]['full_name'].values)
    
    def get_current_product_ids(self) -> List[int]:
        return self.current_product_ids

    def process_message(self, user_input:str) -> None:
        if not self.current_product_ids:
            response = self.category_agent({
                "input": user_input,
                "history": self.memory.load_memory_variables({})["history"]
            })
            generated_products = response["categories"].split(", ")
            best_product_ids = []
            # print(current_products)

            for product in generated_products:
                if len(product) <= 3:
                    continue
                relevant_product_id = self.get_relevant_pruduct_id(user_input, product)
                # print(relevant_product_id)

                try:
                    best_product_id = int(relevant_product_id)
                    best_product_ids.append(best_product_id)
                except:
                    continue

            
            # id лучших товаров - best_product_ids
            # пока вовзвращаю названия,  а так можно id
            print(best_product_ids)

            # print(best_product_ids)

            # print(items_bd['id'].values[:10])
            self.current_product_ids = list(items_bd[items_bd['id'].isin(best_product_ids)]['id'].values)
            self.current_product_ids = [int(product_id) for product_id in self.current_product_ids]
            # self.current_product_ids = [product.strip() for product in self.current_product_ids]
            # print(self.current_product_ids)

            explanation = self.explainer_agent({
                "input": user_input,
                "history": self.memory.load_memory_variables({})["history"],
                "best_products": self.get_full_name_by_id(self.current_product_ids)
            })
            print(f"РЕКОМЕНДУЕМЫЕ ТОВАРЫ:\n------\n {'\n------\n'.join(self.get_full_name_by_id(self.current_product_ids))}")
            print(f"\nОБЪЯСНЕНИЕ:\n {explanation['explanation']}")
            return None
        else:
            response = self.editor_agent({
                "input": user_input,  # Основной ключ для памяти
                "current_products": ", ".join(self.get_product_name_by_id(self.current_product_ids)),
                "history": self.memory.load_memory_variables({})["history"]
            })
            updated_products = [p.strip() for p in response["updated_products"].split("\n")]
            print('Текущие')
            print(self.current_product_ids)
            print('Обновленные (добавила LLM)')
            print(updated_products)

            new_products = list(set(updated_products) - set(self.get_product_name_by_id(self.current_product_ids)))
            id2name = {
                product_id: self.get_product_name_by_id([product_id])[0] for product_id in self.current_product_ids
            }

            updated_product_ids = []
            for product_id, product_name in id2name.items():
                if product_name in updated_products:
                    updated_product_ids.append(product_id)

            # print(id2name)
            # updated_products = list(set(updated_products) & set(self.current_products))

            for product in new_products:
                if len(product) <= 3:
                    continue
                print('Новый продукт:', product)
                relevant_product_id = self.get_relevant_pruduct_id(user_input, product)

                try:
                    best_product_id = int(relevant_product_id)
                    # best_product_name = items_bd[items_bd['id'] == best_product_id]['id'].values[0].strip()
                    updated_product_ids.append(best_product_id)
                except:
                    continue
            print('Итоговый обновленный список (из реальных продуктов)')
            print(updated_product_ids)
            print(self.get_full_name_by_id(updated_product_ids))
            updated_product_ids = [int(product_id) for product_id in updated_product_ids]
            self.current_product_ids = updated_product_ids
            return None


In [111]:
model = OllamaLLM(model="gemma2")


In [112]:
assistant = ShoppingAssistant(model=model, items_bd=items_bd, search_engine=search_engine)

assistant.process_message("Закажи кока колу и рол филадельфия")


[59737, 24311]
РЕКОМЕНДУЕМЫЕ ТОВАРЫ:
------
 Пепси (1 л) 
 Газированный напиток
------
Супер Фила XXXL (58 шт., 1475 г) 
 Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.

ОБЪЯСНЕНИЕ:
 Отличный выбор!  Пепси прекрасно освежит тебя после вкусного ужина из суши "Супер Фила XXXL", в котором есть множество видов роллов, включая твой любимый — филадельфия. 





In [113]:
assistant.current_product_ids

[59737, 24311]

In [114]:
assistant.process_message("Добавь чипсов")

Текущие
[59737, 24311]
Обновленные (добавила LLM)
['Пепси (1 л)', 'Супер Фила XXXL (58 шт., 1475 г)', 'Чипсы', '']
Новый продукт: Чипсы
Итоговый обновленный список (из реальных продуктов)
[59737, 24311, 64734]
['Пепси (1 л) \n Газированный напиток', 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.', 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']


In [ ]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Пепси (1 л) \n Газированный напиток',
 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.',
 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']

In [117]:
assistant.process_message("Добавь парочку сладостей")

Текущие
[59737, 24311, 64734]
Обновленные (добавила LLM)
['Пепси (1 л)', 'Супер Фила XXXL (58 шт., 1475 г)', 'Чипсы куриные Кари (50г)', 'Шоколадные конфеты', 'Желейные карамельки', '']
Новый продукт: Шоколадные конфеты
Новый продукт: Желейные карамельки
Итоговый обновленный список (из реальных продуктов)
[59737, 24311, 64734, 48367, 67621]
['Шоколадный торт с соленой карамелью (1100 г) \n Роскошный шоколадный торт. Толстый слой темного шоколада с соленой карамелью между воздушными шоколадными бисквитами. Украшен мягким шоколадом с фундуком.', 'Итальянский завтрак с карамелью (380 г) \n Нежная пончиковая булочка, запеченная в вафельной форме, подается с нежным карамельно-йогуртовым соусом, жареным арахисом и шариком мороженого.', 'Пепси (1 л) \n Газированный напиток', 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.', 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряны

In [118]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Шоколадный торт с соленой карамелью (1100 г) \n Роскошный шоколадный торт. Толстый слой темного шоколада с соленой карамелью между воздушными шоколадными бисквитами. Украшен мягким шоколадом с фундуком.',
 'Итальянский завтрак с карамелью (380 г) \n Нежная пончиковая булочка, запеченная в вафельной форме, подается с нежным карамельно-йогуртовым соусом, жареным арахисом и шариком мороженого.',
 'Пепси (1 л) \n Газированный напиток',
 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.',
 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']

In [119]:
assistant.process_message("Удали все сладкое")


Текущие
[59737, 24311, 64734, 48367, 67621]
Обновленные (добавила LLM)
['Пепси (1 л)', 'Супер Фила XXXL (58 шт., 1475 г)', 'Чипсы куриные Кари (50г)', '']
Итоговый обновленный список (из реальных продуктов)
[59737, 24311, 64734]
['Пепси (1 л) \n Газированный напиток', 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.', 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']


In [120]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Пепси (1 л) \n Газированный напиток',
 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.',
 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']

In [121]:
assistant.process_message("А добавь какую нибудь пасту итальянскую")


Текущие
[59737, 24311, 64734]
Обновленные (добавила LLM)
['Пепси (1 л)', 'Супер Фила XXXL (58 шт., 1475 г)', 'Чипсы куриные Кари (50г)', 'Паста итальянская', '']
Новый продукт: Паста итальянская
Итоговый обновленный список (из реальных продуктов)
[59737, 24311, 64734, 57925]
['Пепси (1 л) \n Газированный напиток', 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.', 'Паста Фруті де Маре (370г) \n Спагетти, фирменный соус, морепродукты, пармезан', 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']


In [122]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Пепси (1 л) \n Газированный напиток',
 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.',
 'Паста Фруті де Маре (370г) \n Спагетти, фирменный соус, морепродукты, пармезан',
 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']

In [123]:
assistant.process_message("не, давай вместо морепротудуктов пасту с курицей")


Текущие
[59737, 24311, 64734, 57925]
Обновленные (добавила LLM)
['Пепси (1 л)', 'Супер Фила XXXL (58 шт., 1475 г)', 'Чипсы куриные Кари (50г)', 'Паста с курицей', '']
Новый продукт: Паста с курицей
Итоговый обновленный список (из реальных продуктов)
[59737, 24311, 64734, 63964]
['Собака с курицей (350 г) \n Куриное филе, пекинская капуста, морковь, стручковая фасоль, болгарский перец, лук, лапша соба.', 'Пепси (1 л) \n Газированный напиток', 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.', 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']


In [124]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Собака с курицей (350 г) \n Куриное филе, пекинская капуста, морковь, стручковая фасоль, болгарский перец, лук, лапша соба.',
 'Пепси (1 л) \n Газированный напиток',
 'Супер Фила XXXL (58 шт., 1475 г) \n Ролл филадельфия с усами, Ролл калифорния, Макі з маслом, Макі з лососем - 2 шт., Ролл филадельфия с лососем, Чез-ролл.',
 'Чипсы куриные Кари (50г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением дижонской горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических вкусовых добавок.']

In [152]:
search_engine.search('овощи', top_n=5)

[{'description': ' Медовик малиновый Вкусвилл',
  'id': 327592,
  'delievery_time': 102,
  'price': 340},
 {'description': ' Огурцы свежие тепличные среднеплодные, 500г',
  'id': 729214,
  'delievery_time': 91,
  'price': 1197},
 {'description': ' Овсяная каша быстрого приготовления с фруктами, упаковка 5 порций  ',
  'id': 372464,
  'delievery_time': 64,
  'price': 2199},
 {'description': ' Чипсы картофельные со вкусом сыра и зелени в тубе, вес:150г',
  'id': 323068,
  'delievery_time': 25,
  'price': 409},
 {'description': ' Яблоки сорта Гала, свежие и сочные, 1кг',
  'id': 726837,
  'delievery_time': 101,
  'price': 466}]